Milvus is a cloud-based vector database, "Milvus was created in 2019 with a singular goal: store, index, and manage massive embedding vectors generated by deep neural networks and other machine learning (ML) models" (https://milvus.io/docs). Here embedding vectors are used to search in a vector space to reduce time complexity. Milvus applies approximate nearest neighbor (ANN) search to find a top-K similar items. First, data is transformed into vectors using a vector index. Second, a set of nearby vectors is searched for in vector space. The result is the top-K (approximation) of similar items.

Milvus requires the use of Docker Compose and Python version 3.7.1 or later. See the website to get started.

We start with importing required packages.

In [1]:
import pandas as pd
import time
from sklearn.preprocessing import StandardScaler

Subsequently, we import the dataset (https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud).

In [2]:
cc = pd.read_csv("data/creditcard.csv")
cc.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


The aim of vectorisation is properties V1-V28. The 28 features are anonymised using PCA, having a mean of zero. Yet, since the data is unknown each feature should be treated equally in vector space. Therefore, the features are standardised.

In [3]:
cc.iloc[:, 1:29] = StandardScaler().fit_transform(cc.iloc[:, 1:29])
cc.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-0.694242,-0.044075,1.672773,0.973366,-0.245117,0.347068,0.193679,0.082637,0.331128,...,-0.024923,0.382854,-0.176911,0.110507,0.246585,-0.392170,0.330892,-0.063781,149.62,0
1,0.0,0.608496,0.161176,0.109797,0.316523,0.043483,-0.061820,-0.063700,0.071253,-0.232494,...,-0.307377,-0.880077,0.162201,-0.561131,0.320694,0.261069,-0.022256,0.044608,2.69,0
2,1.0,-0.693500,-0.811578,1.169468,0.268231,-0.364572,1.351454,0.639776,0.207373,-1.378675,...,0.337632,1.063358,1.456320,-1.138092,-0.628537,-0.288447,-0.137137,-0.181021,378.66,0
3,1.0,-0.493325,-0.112169,1.182516,-0.609727,-0.007469,0.936150,0.192071,0.316018,-1.262503,...,-0.147443,0.007267,-0.304777,-1.941027,1.241904,-0.460217,0.155396,0.186189,123.50,0
4,2.0,-0.591330,0.531541,1.021412,0.284655,-0.295015,0.071999,0.479302,-0.226510,0.744326,...,-0.012839,1.100011,-0.220123,0.233250,-0.395202,1.041611,0.543620,0.651816,69.99,0


The Python package for Milvus is PyMilvus, several functions need to be imported.

In [4]:
from pymilvus import (connections, utility, FieldSchema, CollectionSchema, DataType, Collection)

First, Milvus requires to specify the field schema, collection schema, and collection name for the database.

In [5]:
#transaction id is an integer represented by dataframe index
#transaction id is primary key for data indexing
transaction_id = FieldSchema(name="transaction_id", dtype=DataType.INT64, is_primary=True)

#transaction features is a float represented by 28 features
transaction_features = FieldSchema(name="transaction_features", dtype=DataType.FLOAT_VECTOR, dim=28)

#collection schema consists of transaction id and features
schema = CollectionSchema(fields=[transaction_id, transaction_features], description="Transaction Similarity Search")

#definition of the collection name
collection_name = "transactions"

Second, we connect to the server.

In [6]:
connections.connect("default", host="localhost", port="19530")

Third, we create a collection to operationalise the database. Two parameters exist:

- **Number of shards [1, 256]**: Number of the shards for the collection to create. Shards divide similarity search into top-K sub-requests and sub-results (https://milvus.io/docs/v2.0.x/data_processing.md). The parameter is for horizontal scaling and results in a trade-off between latency and time complexity.

- **Consistency level ["Strong", "Bounded", "Session", "Eventually", "Customized"]**: consistency of updating data. "Strong" refers to "Milvus will read the most updated data view at the exact time point when a search or query request comes".

In [7]:
def CollectionRefresh(drop, collection_name, sn, cl):

    #if drop the collection is necessary
    if drop == True:

        #if a collection exists, drop the collection
        if utility.has_collection(collection_name) == True:
            utility.drop_collection(collection_name)

        #creation of a collection
        collection = Collection(name=collection_name, schema=schema, using='default',
                                shards_num=sn, consistency_level=cl)
        collection.load()

    return collection

Fourth, we perform a vector search on the database. Three parameters exist:

- **Maximum degree of nodes on each layer of the graph (M) [4, 64]**

- **Search scope of building an index (efC) [8, 512]**

- **Search scope of target retrieval (ef) [k, 32768]**

Besides, the similarity metric is set to the Euclidean distance L2 norms.

In [8]:
def VectorSearch(sn, dn, tk, M, efC, ef, data, output):

    #selection of the data size
    df = data[0:dn]

    #calling the collection refresh function
    collection = CollectionRefresh(drop=True, collection_name="transactions", sn=sn, cl="Strong")

    #registration of the start time for benchmarking
    start = time.time()

    #insertion of the data into the database
    data = [df.index.tolist(), df.iloc[:, 1:29].values.tolist()]
    collection.insert(data)

    #definition of the parameters
    index = {"metric_type":"L2", "index_type":"HNSW", "params":{"M":M, "efConstruction":efC}}
    collection.create_index(field_name="transaction_features", index_params=index)
    search_params = {"metric_type": "L2", "params": {"ef": ef}}

    #extraction of the results
    results = collection.search(data=df.iloc[:, 1:29].values.tolist(), anns_field="transaction_features",
                                param=search_params, limit=(tk+1), expr=None, consistency_level="Strong")

    #registration of the end time for benchmarking
    end = time.time()
    duration = end - start
    print(f"Runtime: {duration}")

    #specification of the output
    if output == "duration":
        return duration
    elif output == "results":
        return results

The vector search is performed.

In [10]:
results = VectorSearch(sn=2, dn=len(cc), tk=256, M=16, efC=32, ef=32,
                       data=cc, output="results")

Runtime: 8701.134235143661


The benchmarking of the time requires the specification of the parameters in a dataframe.

In [9]:
#shards range from 1 up to 128
shard_numbers = [2**exp for exp in range(0, 8)]
shard_numbers = [2**exp for exp in range(4, 5)]
#data ranges from 256 up to 32768 rows
data_sizes = [2**exp for exp in range(8, 18)]
#top-k nodes ranges in 16, 64, 256
top_k = [2**exp for exp in range(8, 14, 2)]
#outcome variable
time_seconds = [None]

#creation of the dataframe
lp1, lp2, lp3, lp4 = pd.core.reshape.util.cartesian_product([shard_numbers, data_sizes, top_k, time_seconds])
bm1 = pd.DataFrame(dict(sn=lp1, dn=lp2, tk=lp3, time=lp4))

The dataframe is accessed and filled by iterating through for-loops.

In [10]:
i = 0

for sn in shard_numbers:
    for dn in data_sizes:
        for tk in top_k:

            print(f"{i}. Number of rows {dn} & shards {sn} & items {tk}")
            bm1.loc[(bm1["sn"] == sn) &
                   (bm1["dn"] == dn) &
                   (bm1["tk"] == tk), "time"] = VectorSearch(sn=sn, dn=dn, tk=tk, M=16, efC=32, ef=32,
                                                             data=cc, output="duration")

            i += 1

The benchmarking of the time requires the specification of the parameters in a dataframe.

In [12]:
#M degrees ranges from 4 up to 64
M_list = [2**exp for exp in range(2, 7)]
#Search scope for search ranges from 1024 up to 32768
ef_list = [2**exp for exp in range(10, 16)]
#Search scope for indexing ranges from 8 up to 512
efC_list = [2**exp for exp in range(3, 10, 2)]
#outcome variable
time_seconds = [None]

#creation of the dataframe
lp1, lp2, lp3, lp4 = pd.core.reshape.util.cartesian_product([M_list, ef_list, efC_list, time_seconds])
bm2 = pd.DataFrame(dict(M=lp1, ef=lp2, efC=lp3, time=lp4))

The dataframe is accessed and filled by iterating through for-loops.

In [13]:
i = 0

for M in M_list:
    for ef in ef_list:
        for efC in efC_list:

            print(f"{i}. Number of M {M} & efC {efC} & ef {ef}")
            bm2.loc[(bm2["M"] == M) &
                   (bm2["ef"] == ef) &
                   (bm2["efC"] == efC), "time"] = VectorSearch(sn=2, dn=32768, tk=1024, M=M, efC=efC, ef=ef,
                                                             data=cc, output="duration")

            i += 1

To store the results into relational format the lists are accessed and expanded to a dataframe in long format. First the lists need to be matched to the corresponding credit card transaction in the original dataframe.

In [11]:
#appending the results from Milvus into a list
ids = []
dists = []

for i in range(len(results)):
    ids.append(results[i].ids)
    dists.append(results[i].distances)

In [12]:
#merging the results to the original dataframe
cc_n = cc[0:len(results)]
cc_n = cc_n.copy()
cc_n["Distances"] = dists
cc_n["Relationships"] = ids
cc_n.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,Distances,Relationships
0,0.0,-0.694242,-0.044075,1.672773,0.973366,-0.245117,0.347068,0.193679,0.082637,0.331128,...,-0.176911,0.110507,0.246585,-0.392170,0.330892,-0.063781,149.62,0,"[0.0, 1.5763095617294312, 2.1485962867736816, ...","[0, 107821, 41901, 22656, 96012, 66118, 91891,..."
1,0.0,0.608496,0.161176,0.109797,0.316523,0.043483,-0.061820,-0.063700,0.071253,-0.232494,...,0.162201,-0.561131,0.320694,0.261069,-0.022256,0.044608,2.69,0,"[0.0, 2.315798610652564e-07, 9.64720493357163e...","[1, 21737, 22566, 29318, 131025, 140707, 11125..."
2,1.0,-0.693500,-0.811578,1.169468,0.268231,-0.364572,1.351454,0.639776,0.207373,-1.378675,...,1.456320,-1.138092,-0.628537,-0.288447,-0.137137,-0.181021,378.66,0,"[0.0, 4.630891799926758, 6.204038143157959, 6....","[2, 93003, 637, 65173, 111709, 60244, 69693, 8..."
3,1.0,-0.493325,-0.112169,1.182516,-0.609727,-0.007469,0.936150,0.192071,0.316018,-1.262503,...,-0.304777,-1.941027,1.241904,-0.460217,0.155396,0.186189,123.50,0,"[0.0, 0.9781743288040161, 1.9065601825714111, ...","[3, 44225, 27394, 59386, 169738, 77298, 47662,..."
4,2.0,-0.591330,0.531541,1.021412,0.284655,-0.295015,0.071999,0.479302,-0.226510,0.744326,...,-0.220123,0.233250,-0.395202,1.041611,0.543620,0.651816,69.99,0,"[0.0, 3.350313663482666, 3.942937135696411, 4....","[4, 138045, 97786, 104325, 78062, 38451, 19721..."


The structure of relationships in a list is unfavourable and the results need to be altered to a long format. The JSON package is helpful for expanding the string structure into integers and floats. The new format is again saved in the original format.

In [14]:
import json

lists = []

for i in range(0, len(cc)):
    string_list = cc_n.Relationships[i]
    lists.append(json.loads(string_list))

cc_n["Relationships"] = lists

lists = []

for i in range(0, len(cc)):
    string_list = cc_n.Distances[i]
    lists.append(json.loads(string_list))

cc_n["Distances"] = lists

In [15]:
#storing the results into a CSV file
cc_n.to_csv("data/cc_all_s2_HNSW_M16_ef32_efc32_k256_v2.csv")

In [16]:
#formatting to a long list
long_list = []

for i in range(0, len(cc.Relationships)):
    for j in range(0, len(cc.Relationships[i])):
        #from_id, distance, priority, to_id, type
        long_list.append([i, cc.Distances[i][j], j, cc.Relationships[i][j], "SIMILAR_TO"])

relations = pd.DataFrame(long_list)
relations.to_csv("data/relations.csv", header=False, index=False)

We safe the relationships for access later on. The structure is as following.

In [14]:
df_rel = pd.read_csv("data/relations.csv", header=None, names=(["from_id", "distance", "priority", "to_id", "type"]))
df_rel.head()

,from_id,distance,priority,to_id,type
0,0,0.000000,0,0,SIMILAR_TO
1,0,1.576310,1,107821,SIMILAR_TO
2,0,2.148596,2,41901,SIMILAR_TO
3,0,2.814823,3,22656,SIMILAR_TO
4,0,2.918754,4,96012,SIMILAR_TO


The inverse of the distance is the similarity between nodes. Infinite values refer to the similarity of a node to itself. The similarity is set to 0 because it is not relevant. The node is excluded from queries in Neo4j later on.

In [15]:
inverse = 1/df_rel["distance"]
df_rel.insert (2, "inverse", inverse)
df_rel.head()

,from_id,distance,inverse,priority,to_id,type
0,0,0.000000,inf,0,0,SIMILAR_TO
1,0,1.576310,0.634393,1,107821,SIMILAR_TO
2,0,2.148596,0.465420,2,41901,SIMILAR_TO
3,0,2.814823,0.355262,3,22656,SIMILAR_TO
4,0,2.918754,0.342612,4,96012,SIMILAR_TO


In [19]:
from numpy import inf
df_rel[df_rel["inverse"] == inf] = 0
df_rel.head()

,from_id,distance,inverse,priority,to_id,type
0,0,0.000000,0.000000,0,0,0
1,0,1.576310,0.634393,1,107821,SIMILAR_TO
2,0,2.148596,0.465420,2,41901,SIMILAR_TO
3,0,2.814823,0.355262,3,22656,SIMILAR_TO
4,0,2.918754,0.342612,4,96012,SIMILAR_TO


The final dataset is saved, the CSV may not have headers and indices due to the structure for admin import in Neo4j.

In [20]:
df_rel.to_csv("data/relations.csv", header=False, index=False)